# CH02. Model
---
장고는 모델을 이용하여 데이터베이스를 처리한다. 보통 데이터베이스에 데이터를 저장하고 조회하기 위해서 SQL 쿼리문을 이용해야 하지만 장고의 모델(Model)을 사용하면 이런 SQL 쿼리문의 도움없이 데이터를 쉽게 처리할 수 있다.<br>
### 1. 장고 앱 migrate
<strong>1.1. config/setting.py 뜯어보기</strong><br>
```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]
```
위에 있는 앱들은 모두 장고 프로젝트 생성시 기본적으로 설치되는 앱들이다. <br><br>

config/setting.py 파일에는 설치된 앱 뿐만 아니라 사용하는 데이터베이스에 대한 정보도 정의되어 있다.
```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}
```
데이터베이스 엔진은 ``` django.db.backends.sqlite3 ```라고 정의되어 있다. sqlite는 파일 기반의 아주 작은 데이터베이스이다. 그리고 데이터베이스 파일은 BASE_DIR 하위에 db.sqlite3 라는 파일에 저장한다고 정의되어 있다. BASE_DIR은 프로젝트 디렉토리를 의미한다. 이 또한 setting.py 상단에 정의되어 있다.<br><br>

<strong>1.2. migrate로 앱들이 필요로 하는 테이블 생성하기</strong><br>
방법은 다음과 같다.<br>
```shell
(virenv) ...\project> python manage.py migrate
```
결과는 다음과 같다.
```shell
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying sessions.0001_initial... OK

(mysite) C:\projects\mysite> 
```
migrate를 수행하면 admin, auth, contenttypes, sessions 앱들이 사용하는 테이블들이 생성된다. 어떤 테이블들이 생성되는지 알기 귀찮다. 테이블을 직접 건드릴 일이 없단다. 장고의 ORM(Object Relational Mapping) 기능을 사용하여 쿼리문을 몰라도 데이터 작업을 쉽고 편리하게 할 수 있다.

><strong> ORM의 장점 </strong><br>
>MySQL에서 오라클로 데이터베이스가 변경된다면 프로그램에서 사용한 쿼리문을 모두 해당 데이터베이스의 규칙에 맞게 수정해야 하는 유지보수적 측면에서 치명적인 문제와 시스템의 성능을 저하시키는 상황이 발생하지 않는다. 쿼리문이 아니라 모델을 사용하기 때문에 프로그램을 전혀 수정할 필요가 없다.

<strong>1.3. 모델 작성하기</strong><br>
myapp이 사용할 데이터 모델을 만들어 보자.<br>
질문(Question) 모델을 만들 건데 다음과 같은 속성이 필요하다.
- subject : 질문의 제목
- content : 질문의 내용
- create_date : 질문을 작성한 일시
마찬가지로 답변(Answer) 모델에는 다음과 같은 속성이 필요하다.
- question : 어떤 질문인지 알아야한다.
- content : 답변의 내용
- create_date : 답변을 작성한 일시
<br><br>
이러한 모델을 myapp/model.py에 작성한다.<br><br>

``` myapp/model.py ```
```python
from django.db import models # 기본 db모델에 관한 클래스 모듈이다.

class Question(models.Model): # 다음과 같이 Model 클래스의 상속을 받아야한다.
    subject = models.CharField(max_length=200) # 문자열은 캐릭터 필드를 사용한다. 최대 길이 200
    content = models.TextField() # 굉장히 긴 문자열은 텍스트 필드를 사용한다.
    create_date = models.DateTimeField() # 시각의 경우는 이러한 필드를 사용한다.
    
class Answer(models.Model):
    question = model.ForeignKey(Question, on_delete=model.CASCADE) # 아래에서 설명한다.
    content = model.TextField()
    create_date = models.DateTimeField()
```
create_date 처럼 날짜와 시간에 관계된 속성은 DateTimeField를 사용해야 한다.<br>
Answer 모델은 질문에 대한 답변에 해당되므로 Question 모델을 속성으로 가져야한다. <strong>기존 모델을 속성으로 가져갈 경우 </strong>``` ForeignKey ```를 이용해야 한다. ``` ForeignKey ```는 다른 모델과의 연결을 의미한다. <br>
이때 두 번째 파라미터의 ``` on_delete=models.CASCADE ```의 의미는 이 답변과 연결된 질문(Question)이 삭제될 경우 답변(Answer)도 함께 삭제된다는 의미이다.<br><br>
그 외에도 장고에서 사용하는 속성은 다음과 같이 많이있다.
[https://docs.djangoproject.com/en/3.0/ref/models/fields/#field-types](https://docs.djangoproject.com/en/3.0/ref/models/fields/#field-types)<br>
다 못적겠으니 링크를 두겠다.<br><br>

<strong>1.4. 모델 생성하기</strong><br>
모델의 골격이 잡혔기 때문에 모델을 생성해보자. ``` migrate ```명령을 수행해야 한다. 그전에 ``` makemigrations ```명령을 먼저 수행함을 잊지말자.<br><br>
```shell
(virenv) >python manage.py makemigrations
Migrations for 'myapp':
    myapp/migrations/0001_initial.py
        - Create model Question
        - Create model Answer
        
(virenv) >
```
``` makemigrations ``` 명령어는 모델을 생성한거나 모델에 변화가 있을경우에 실행해 주어야 하는 명령어이다. 위 명령을 수행하면 0001_initial.py라는 파이썬 파일이 자동으로 생성된다. <br><br>

>makemigrations를 수행했더라도 실제로 테이블이 생성되지는 않는다. 테이블을 실제 생성하는 명령어는 migrate 명령을 통해서만 가능하다.
<br>

<strong>1.5. sqlmigrate</strong><br>
migrate 실행시 실제 어떤 쿼리문이 실행되는지 sqlmigrate 명령을 이용하여 확인할 수 있다.
```shell
>python manage.py sqlmigrate myapp 0001
BEGIN;
--
-- Create model Question
--
CREATE TABLE "pybo_question" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "subject" varchar(200) NOT NULL, "content" text NOT NULL, "create_date" datetime NOT NULL);
--
-- Create model Answer
--
CREATE TABLE "pybo_answer" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "content" text NOT NULL, "create_date" datetime NOT NULL, "question_id" integer NOT NULL REFERENCES "pybo_question" ("id") DEFERRABLE INITIALLY DEFERRED);
CREATE INDEX "pybo_answer_question_id_e174c39f" ON "pybo_answer" ("question_id");
COMMIT;
```

<br><br>
<strong>1.6. migrate</strong><br>
이제 migrate 명령을 수행하여 실제 테이블을 생성하도록 하자.
```shell
>python manage.py migrate
```
이때 실제 테이블은 myapp_question, myapp_answer이라는 이름으로 생성된다.

<strong>1.7. 장고 쉘을 사용하여 DB 조작하기</strong><br>
일반적인 파이썬 쉘을 실행하는 것이 아니라 장고 쉘을 실행해야 한다. 장고 쉘을 실행하는 명령어는 다음과 같다.<br>
``` python manage.py shell ```<br>

- 모델을 import 하여 생성하기
```python 
>>> from myapp.models import Question, Answer # 모델 가져오기
>>> q = Question(subject='질문1', content='첫 번째 질문입니다.', create_date=timezone.now()) # 쉘에서 데이터 생성
>>> q.save() # 테이블로 업로드
```
- 생성된 객체의 id값 확인하기
```python
>>> q.id
1
```
- 생성된 모든 객체의 데이터 조회하기
```python
>>> Question.objects.all()
<QuerySet [<Question: Question object (1)>]>
```
저장한 Question 모델의 데이터는 ``` Question.objects ```를 통해서 조회할 수 있다.<br>
``` Question.objects.all() ```은 Question에 저장된 모든 데이터를 조회하는 함수이다. 결과값으로는 QuerySet 객체가 리턴되는데 위처럼 2개의 Question객체를 포함하고 있다.

- 모델의 문자열 속성값을 추가하여 id대신 제목을 표시하도록 할 수 있다.(Trivial)
```python
class Question(models.Model):
    ...
    def __str__(self):
        return self.subject
    ...
```
- 장고 쉘 종료하기
```python
quit()
```
- 생성된 특정 객체의 데이터 조회하기
```python
>>> Question.objects.filter(id=1) # 해당 조건을 만족하는 모든 데이터를 QuerySet으로 리턴
<QuerySet [<Question: 질문1>]>
>>> Question.objects.get(id=1) # 해당 조건을 만족하는 한 개의 데이터를 객체로 리턴
<Question: 질문1>
# 데이터를 찾을 때 사용하는 속성은 id 속성말고도 장고에서 기본으로 제공하는
# __contains가 있다. 사용 예시는 다음과 같다.
# .filter(subject__contains='장고')
```
- 특정 데이터 수정하기
```python
>>> q = Question.objects.get(id=1)
>>> q
<Question: 질문1>
>>> q.subject = '수정된 질문1'
>>> q.save()
>>> q
<Question: 수정된 질문1>
```
- 특정 데이터 삭제하기
```python
>>> q = Question.objects.get(id=1)
>>> q.delete()
(1, {'myapp.Answer': 0, 'myapp.Question': 1})
```
delete를 수행하면 해당 데이터가 삭제된다. 위의 의미는 Answer객체 0개, Question객체 1개가 삭제되었다는 것을 의미한다.
- ForignKey로 연결된 객체의 작성
```python
>>> q = Question.objects.get(id=1)
from django.utils import timezone
>>> a = Answer(question=q, content='생성확인', create_date=timezone.now())
>>> a.save()
```
- ForignKey로 연결된 객체 검색
```python
>>> q.answer_set.all()
```
이는 Answer이 Question과 연결되어 있기 때문에 ``` answer_set ```을 사용할 수 있다. 이로인해 질문과 연결된 답변을 가져올 수 있다. 물론 Question 하나에는 여러개의 Answer이 연결될 수 있기 때문에 answer_set이 가능한데, question_set은 상식적으로 안된다. 오직 가능한 것은 ``` a.question ```이다.

<strong>Appendix. GUI로 데이터 테이블 조작하기</strong><br>
 위에서 장고 쉘에서 조작한 방식들은 사실 admin에서 GUI로 쉽게 조작가능하다.